# Plot empirical ARI curves for TC wind observations

Here, we plot the empirical average recurrence intervals for wind observations that coincide with the passage of a tropical cyclone. The observations are extracted from the daily maximum wind gust observation records, when there is a TC (in the IBTrACS database) within 200 km of the station. If there are multiple days with the same TC in the vicinity, we take the highest of the daily maximum wind gusts observed during the period. 

We assume the length of record corresponds to the number of years for which data is available in the station record (i.e. with digitised observations). Many stations have been open for longer than for which data is presently available. 

In [1]:
%matplotlib inline
from __future__ import print_function, division

In [2]:
import os
import io
import sys
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import statsmodels.api as sm
from datetime import datetime

from scipy.stats import genpareto, scoreatpercentile, lognorm


from extremes import returnLevels, empReturnPeriod, returnPeriodUncertainty, gpdSelectThreshold
from distributions import fittedPDF

# Import widgets for interactive notebook
from ipywidgets import interact, fixed
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

c:\python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def parseTimeString(yr, month, day, timestr):
    """
    Parse year, month and day as strings and return a datetime.
    
    Handles the case of a missing time string (Pandas returns nan 
    if the field is empty).
    """
    if timestr is np.nan:
        timestr='0000'
    formattime = '{0}-{1}-{2} {3}'.format(yr, month, day, time)
    
    return datetime.strptime(formattime, '%Y-%m-%d %H%M')

def convertStartDate(datestr):
    if datestr == '':
        return ""
    dtstr = "01/{0}".format(datestr)
    return datetime.strptime(dtstr, '%d/%m/%Y')

def parseObsDate(year, month, day, hour, minute):
    timestr = "{0}-{1}-{2} {3}:{4}".format(year, month, day, hour, minute)
    return datetime.strptime(timestr, '%Y-%m-%d %H:%M')

In [4]:
STNTYPES = [('st', 'S2'), ('stnId', 'i'), ('stnDistCode', 'S4'), ('stnName', 'S'), 
            ('stnDateOpen', 'S10'), ('stnDateClosed', 'S10'), ('stnLat', 'f8'), 
            ('stnLon', 'f8'), ('method', 'S15'), ('state', 'S3'), 
            ('stnElevation', 'f8'), ('baroElev', 'i'), ('stnWMONumber', 'i'), ('stnDataStart', 'i'), 
            ('stnDataEnd', 'i'), ('blank', 'S3'), ('percentcomplete', 'f8'), ('pcqualy', 'f8'), 
            ('pcqualn', 'f8'), ('pcqualw', 'f8'), ('pcquals', 'f8'), ('pcquali', 'f8'), ('end', 'S1')]
STNCONVERT = {'stnName' : str.rstrip}

In [8]:
stationFilePath = "C:/WorkSpace/data/raw/daily_max_wind_gust/"
stnfile = pjoin(stationFilePath, "DC02D_StnDet_999999999425050.txt")

locations = pd.read_csv(stnfile, parse_dates=[4, 5],
                        usecols=(1,2,3,4,5,6,7,9,10,12,13,14,16), 
                        names = np.dtype(STNTYPES).names,
                        skiprows=1, engine='python', index_col='stnId', 
                        converters=STNCONVERT)
locationNameList = list(locations['stnName'])

locations.head(10)

,stnDistCode,stnName,stnDateOpen,stnDateClosed,stnLat,stnLon,state,stnElevation,stnWMONumber,stnDataStart,stnDataEnd,percentcomplete
stnId,,,,,,,,,,,,
1006,01,WYNDHAM AERO,01/1951,,-15.5100,128.1503,WA,3.8,95214,2003,2017,96
1007,01,TROUGHTON ISLAND,09/1956,,-13.7542,126.1485,WA,6.0,94102,2008,2017,94
1009,01,KURI BAY,08/1961,09/2012,-15.4875,124.5222,WA,12.0,,2001,2001,100
1019,01,KALUMBURU,11/1997,,-14.2964,126.6453,WA,23.0,94100,2003,2017,93
1020,01,TRUSCOTT,01/1944,,-14.0900,126.3867,WA,51.0,95101,2004,2017,93
2012,02,HALLS CREEK METEOROLOGICAL OFFICE,01/1944,,-18.2292,127.6636,WA,422.0,99201,1962,2017,94
2056,02,KUNUNURRA AERO,09/1971,,-15.7814,128.7100,WA,44.0,94216,1994,2017,97
2064,02,ARGYLE AERODROME,01/1986,,-16.6381,128.4517,WA,164.0,94217,2002,2017,99
3003,03,BROOME AIRPORT,01/1939,,-17.9475,122.2353,WA,7.4,94203,1941,2017,99


In [15]:
def loadObservations(locId):
    names = ['recid', 'stnId', 'datetime', 'gust',
             'direction', 'quality', 'cycName']
    obsPath = "C:/WorkSpace/data/derived/tcobs/daily"
    filename = pjoin(obsPath, "bom_{0:06d}.csv".format(locId))
    try:
        obsdf = pd.read_csv(filename, skiprows=1, names=names,
                            parse_dates=[2], infer_datetime_format=True)
    except FileNotFoundError:
        print("No data file for stnId: {0}".format(locId))
        return None
    return obsdf

def getStationDates(locId):
    startYear = locations.loc[locId]['stnDataStart']
    endYear = locations.loc[locId]['stnDataEnd']
    numYears = endYear - startYear + 1
    return numYears

-

In [16]:
def plotCurve(locationName):
    fig, ax = plt.subplots(1, 1)
    locId = locations.index[locationNameList.index(locationName)]
    plotObservedHazard(locId, ax)
    plt.show()

interact(plotCurve, locationName=locationNameList)

<function __main__.plotCurve>